In [3]:
import seaborn as sns

#titanic 데이터셋 가져오기
df=sns.load_dataset("titanic")
display(df.head())
print(df.info())

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


## 누락 데이터 확인

In [24]:
print(df["deck"].value_counts(dropna=False)) # dropna를 기입하지 않으면 nan 개수를 반환하지 않음
print(df["deck"].value_counts())
print(df["deck"].count()) # 그냥 count 는 열별로 세줌(nan 제외)
print(df.head().isnull()) #누락 데이터면 True, 유효한 데이터면 False 반환
print(df.head().notnull()) #유효한 데이터면 False, 누락 데이터면 FAlse 반환
print(df.head().isnull().sum(axis=0)) # axis=0을 해주어야 열별 공백값 반환

NaN    688
C       59
B       47
D       33
E       32
A       15
F       13
G        4
Name: deck, dtype: int64
C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: deck, dtype: int64
203
   survived  pclass    sex    age  sibsp  parch   fare  embarked  class  \
0     False   False  False  False  False  False  False     False  False   
1     False   False  False  False  False  False  False     False  False   
2     False   False  False  False  False  False  False     False  False   
3     False   False  False  False  False  False  False     False  False   
4     False   False  False  False  False  False  False     False  False   

     who  adult_male   deck  embark_town  alive  alone  
0  False       False   True        False  False  False  
1  False       False  False        False  False  False  
2  False       False   True        False  False  False  
3  False       False  False        False  False  False  
4  False       False   True        False  False  False  
   surviv

In [26]:
df.index

RangeIndex(start=0, stop=891, step=1)

## 누락 데이터 제거

### for 문 이용

In [40]:
#for 문 이용
missing_df = df.isnull()
for col in missing_df.columns:
    missing_count = missing_df[col].value_counts() # False의 값을 반환
    try:
        print(col,":",missing_count[True]) # True 행이 있으면 반환, 즉 null 값이 있으면 그 개수 반환
    except:
        print(col,":",0)

survived : 0
pclass : 0
sex : 0
age : 177
sibsp : 0
parch : 0
fare : 0
embarked : 2
class : 0
who : 0
adult_male : 0
deck : 688
embark_town : 2
alive : 0
alone : 0


### dropna 이용

In [74]:
# Nan 값이 500 개 이상인 열을 모두 삭제
df_thresh = df.dropna(axis=1) #누락값이 존재하는 칼럼 제거
df_thresh = df.dropna(axis=1, thresh=500) #유효값이 500개 이상인 칼럼 제거
print(len(df_thresh.columns))
print(len(df.columns))

# age 열에 나이 데이터가 없는 모든 행 삭제
df_age = df.dropna(how="all", axis=0) # 모든 값이 NaN인 인덱스 삭제
df_age = df.dropna(how="any", axis=1) # 하나라도 NaN이 있는 칼럼 삭제
print(df_age.shape)
df_age = df.dropna(how="all", axis=1) # 모든 값이 NaN 인 칼럼 삭제
print(df_age.shape)
df_age = df.dropna(subset=["age","deck"], axis=0, how="any") # age,deck 칼럼 값 중에 NaN 이 하나라도 존재하는 index 삭제
print(df_age.shape)
df_age = df.dropna(subset=["age","deck"], axis=0, how="all") # age,deck 칼럼 값이 모두 NaN 인 index 삭제
print(df_age.shape)

14
15
(891, 11)
(891, 15)
(184, 15)
(733, 15)


## 누락 데이터 치환

In [75]:
mean_age = df["age"].mean(axis=0)
df["age"].fillna(mean_age, inplace=True)

In [85]:
print(df["embark_town"][825:830])
most_freq = df["embark_town"].value_counts().idxmax() # 가장 value 값이 큰 index 반환
print(most_freq,"\n")

df["embark_town"].fillna(most_freq, inplace=True)
print(df["embark_town"][825:830])

print(df["deck"].fillna(method="bfill")) # NaN 이 있는 행의 다음 행에 있는 값으로 바꿔줌
print(df["deck"].fillna(method="ffill")) # NaN 이 있는 행의 직전 행에 있는 값으로 바꿔줌

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829            NaN
Name: embark_town, dtype: object
Southampton 

825     Queenstown
826    Southampton
827      Cherbourg
828     Queenstown
829    Southampton
Name: embark_town, dtype: object


# 중복 데이터

In [101]:
import pandas as pd
import numpy as np
df = pd.DataFrame({"c1":"a a b a b".split(), "c2":[1,1,1,3,1], "c3":[1,1,2,3,1]})
print(df)
print(df.duplicated()) # 처음 나오는 값은 False
print(df["c3"].duplicated())
print(df["c2"].duplicated(keep="first")) # 처음 나오는 값을 False 로 반환
print(df["c2"].duplicated(keep="last")) # 마지막에 나오는 값을 False로 반환
print(df["c2"].duplicated(keep=False)) # 중복값이면 무조건 True 반환


print(df.drop_duplicates()) # inplace = False 가 기준값
print(df.drop_duplicates(subset=["c2"])) # c2 열을 기준으로 중복된 행을 제거
print(df.drop_duplicates(subset=["c2","c3"])) # c2와 c3 열을 기준으로 두 개의 열 모두 중복값일 때 제거

  c1  c2  c3
0  a   1   1
1  a   1   1
2  b   1   2
3  a   3   3
4  b   1   1
0    False
1     True
2    False
3    False
4    False
dtype: bool
0    False
1     True
2    False
3    False
4     True
Name: c3, dtype: bool
0    False
1     True
2     True
3    False
4     True
Name: c2, dtype: bool
0     True
1     True
2     True
3    False
4    False
Name: c2, dtype: bool
0     True
1     True
2     True
3    False
4     True
Name: c2, dtype: bool
  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   3   3
4  b   1   1
  c1  c2  c3
0  a   1   1
3  a   3   3
  c1  c2  c3
0  a   1   1
2  b   1   2
3  a   3   3


# 데이터 표준화

In [103]:
df = pd.read_csv("./auto-mpg.csv")
print(df.head(3))

mpg_to_kpl = 1.60934/3.78541
df["kpl"] = df["mpg"] * mpg_to_kpl
print(df.head(3))

df["kpl"]=df["kpl"].round(2) # 소숫점 둘째자리까지 나타냄. 셋째자리에서 반올림
print(df.kpl.head(3))

    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  15.0          8         350.0        165    3693          11.5          70   
2  18.0          8         318.0        150    3436          11.0          70   

   origin                    carname  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
    mpg  cylinders  displacement horsepower  weight  acceleration  model year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  15.0          8         350.0        165    3693          11.5          70   
2  18.0          8         318.0        150    3436          11.0          70   

   origin                    carname       kpl  
0       1  chevrolet chevelle malibu  7.652571  
1       1          buick skylark 320  6.377143  
2       1         plymouth satellite  7.652571 

In [115]:
print(df["horsepower"].dtype)

# "?" 인자 제거
print(df["horsepower"].unique()) # df["horsepower"] 의 고유값 확인
df["horsepower"].replace("?",np.nan,inplace=True)
df.dropna(subset=["horsepower"], axis=0, inplace=True)

# 문자열을 astype을 이용해서 실수형으로 변환
df["horsepower"] = df["horsepower"].astype("float")
print(df["horsepower"].dtype)

# 정수형을 문자열로 저장
print(df["origin"].unique())
print(df["origin"].dtypes)

df["origin"].replace({1:"USA",2:"EU",3:"JAP"}, inplace=True)
print(df["origin"].unique())
print(df["origin"].dtypes)

#문자열을 범주형으로 변환
df["origin"]=df["origin"].astype("category")
print(df["origin"].dtypes)
df["origin"]=df["origin"].astype("str")
print(df["origin"].dtypes)

float64
[130. 165. 150. 140. 198. 220. 215. 225. 190. 170. 160.  95.  97.  85.
  88.  46.  87.  90. 113. 200. 210. 193. 100. 105. 175. 153. 180. 110.
  72.  86.  70.  76.  65.  69.  60.  80.  54. 208. 155. 112.  92. 145.
 137. 158. 167.  94. 107. 230.  49.  75.  91. 122.  67.  83.  78.  52.
  61.  93. 148. 129.  96.  71.  98. 115.  53.  81.  79. 120. 152. 102.
 108.  68.  58. 149.  89.  63.  48.  66. 139. 103. 125. 133. 138. 135.
 142.  77.  62. 132.  84.  64.  74. 116.  82.]
float64
<StringArray>
['USA', 'JAP', 'EU']
Length: 3, dtype: string
string
<StringArray>
['USA', 'JAP', 'EU']
Length: 3, dtype: string
string
category
object


## 구간 분할

### np.histogram(), pd.cut()

In [126]:
df = pd.read_csv("./auto-mpg.csv")
df["horsepower"].replace({"?":np.nan},inplace=True)
df.dropna(subset=["horsepower"],axis=0, inplace=True)
df["horsepower"]=df["horsepower"].astype("float")

print(df["horsepower"].head(10))

count, bin_dividers = np.histogram(np.arange(1,10),bins=3) # bins=3 을 통해 3개의 구간 설정
print(count) # 하나의 구간에 들어가는 개수
print(bin_dividers) # 경계값, 3개의 구간으로 나누기 때문에 경계값 4개

count, bin_dividers = np.histogram(df["horsepower"],bins=3)
print(count)
print("\n")
print(bin_dividers)

# 나이
ages = [2,10,21,23,32,41,50,61]

bin_points = [2,20,30,50,70] # 경계값
bin_names = ["미성년자","청년","중년","노년"]

pd.cut(ages, bins=bin_points, labels=bin_names, include_lowest=True) # 첫 경계값 포함은 앞의 구간에 <= 인 것.

#3개의 구간에 이름 지정
bin_names=["저출력","보통출력","고출력"]
df["hp_bin"]=pd.cut(df["horsepower"],bins=bin_dividers,labels=bin_names,include_lowest=True)
print(df[["horsepower","hp_bin"]]) # bins에서 바로 3을 입력해도 가능

0    130.0
1    165.0
2    150.0
3    150.0
4    140.0
5    198.0
6    220.0
7    215.0
8    225.0
9    190.0
Name: horsepower, dtype: float64
[3 3 3]
[1.         3.66666667 6.33333333 9.        ]
[257 103  32]


[ 46.         107.33333333 168.66666667 230.        ]
     horsepower hp_bin
0         130.0   보통출력
1         165.0   보통출력
2         150.0   보통출력
3         150.0   보통출력
4         140.0   보통출력
..          ...    ...
393        86.0    저출력
394        52.0    저출력
395        84.0    저출력
396        79.0    저출력
397        82.0    저출력

[392 rows x 2 columns]


## 더미 변수
- pd.get_dummies()
- sklearn
    1. 원핫벡터로 변환 -> preprocessing.LabelEncoder()
    1. 희소행렬 -> preprocessing.OneHotEncoder()
    1. numpy 배열로 바꿔줌

In [128]:
print(pd.get_dummies(df["hp_bin"]))

,저출력,보통출력,고출력
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
393,1,0,0
394,1,0,0
395,1,0,0
396,1,0,0


In [132]:
from sklearn import preprocessing
items = ["tv","냉장고","전자렌지","컴퓨터","선풍기","선풍기","믹서","믹서"]

# 전처리를 위한 label_encoder 라는 객체 생성
# 해당 객체를 통해서 범주형 데이터를 변형
label_encoder = preprocessing.LabelEncoder()

onehot_labeled = label_encoder.fit_transform(items)
print(onehot_labeled)

onehot_reshaped = onehot_labeled.reshape(len(onehot_labeled),1)
print(onehot_reshaped)
# 희소 행렬로 변환
onehot_encoder = preprocessing.OneHotEncoder()
onehot_fitted = onehot_encoder.fit_transform(onehot_reshaped)
print(onehot_fitted)

# numpy 배열로 전환
print(onehot_fitted.toarray())

[0 1 4 5 3 3 2 2]
[[0]
 [1]
 [4]
 [5]
 [3]
 [3]
 [2]
 [2]]
  (0, 0)	1.0
  (1, 1)	1.0
  (2, 4)	1.0
  (3, 5)	1.0
  (4, 3)	1.0
  (5, 3)	1.0
  (6, 2)	1.0
  (7, 2)	1.0
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


# 데이터 정규화
-사전단계 : 데이터 표준화 -> 이상치가 있을 경우 표준화를 통해 outlier을 제거
1. 각 열의 데이터를 해당 열의 최대값의 절댓값으로 나누는 방법
1. 각 열의 데이터 중에서 최대값과 최소값을 뺀 값으로 나누는 방법

In [134]:
# 표준화
print(df["horsepower"].describe())

print((df.horsepower-df.horsepower.mean())/df.horsepower.std())

count    392.000000
mean     104.469388
std       38.491160
min       46.000000
25%       75.000000
50%       93.500000
75%      126.000000
max      230.000000
Name: horsepower, dtype: float64
0      0.663285
1      1.572585
2      1.182885
3      1.182885
4      0.923085
         ...   
393   -0.479835
394   -1.363154
395   -0.531795
396   -0.661694
397   -0.583754
Name: horsepower, Length: 392, dtype: float64


In [140]:
# 정규화사용법1
print(df["horsepower"]/abs(df["horsepower"].max()))
print(df["horsepower"].describe())

0      0.565217
1      0.717391
2      0.652174
3      0.652174
4      0.608696
         ...   
393    0.373913
394    0.226087
395    0.365217
396    0.343478
397    0.356522
Name: horsepower, Length: 392, dtype: float64
count    392.000000
mean       0.454215
std        0.167353
min        0.200000
25%        0.326087
50%        0.406522
75%        0.547826
max        1.000000
Name: horsepower, dtype: float64


In [ ]:
# 정규화 사용법2
min_x = df.horsepower - df.horsepower.min()
min_max = df.horsepower.max()-df.horsepower.min()
df.horsepower = min_x/min_max
print(df.horsepower.describe())


# 시계열
- 자료형 변환
    - 타임스탬프로 변환 -> pd.to_datetime()
    - 타임스탬프를 기간으로 변환 - >to_period(freq="D"/"M"/"A")
    
- 시계열 데이터 만들기
    - 타임스탬프 데이터 : pd.date_range(start,end,periods,freq,tz)
    - 기간 데이터 : pd.date_period

In [154]:
import pandas as pd
stock=pd.read_csv("./stock-data.csv")
print(stock.info())
stock["new_Date"]=pd.to_datetime(stock["Date"])
print(stock.info())
print("Date")
#새로운 인덱스 지정
stock.set_index("new_Date",inplace=True) # 행을 index로 지정 후, 알아서 원래 행 삭제해줌
stock.drop(["Date"],inplace=True, axis=1)
print(stock.head(2))
print(stock.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20 non-null     object
 1   Close   20 non-null     int64 
 2   Start   20 non-null     int64 
 3   High    20 non-null     int64 
 4   Low     20 non-null     int64 
 5   Volume  20 non-null     int64 
dtypes: int64(5), object(1)
memory usage: 1.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      20 non-null     object        
 1   Close     20 non-null     int64         
 2   Start     20 non-null     int64         
 3   High      20 non-null     int64         
 4   Low       20 non-null     int64         
 5   Volume    20 non-null     int64         
 6   new_Date  20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(5), ob

In [157]:
dates = ["2019-01-01","2020-03-01","2021-06-01"]

ts_dates = pd.to_datetime(dates)
print(ts_dates)

#Timestamp를 Period로 변환
pr_day = ts_dates.to_period(freq="D") #1일 기간
print(pr_day)
pr_day = ts_dates.to_period(freq="M") #1개월
print(pr_day)
pr_year = ts_dates.to_period(freq="A") # 1년기간
print(pr_year)

DatetimeIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='datetime64[ns]', freq=None)
PeriodIndex(['2019-01-01', '2020-03-01', '2021-06-01'], dtype='period[D]', freq='D')
PeriodIndex(['2019-01', '2020-03', '2021-06'], dtype='period[M]', freq='M')
PeriodIndex(['2019', '2020', '2021'], dtype='period[A-DEC]', freq='A-DEC')


### 배열 생성

In [170]:
ts_ms = pd.date_range(start="2019-01-01",end=None, periods=6,freq="MS",tz="Asia/Seoul") # 각 달의 시작을 기준으로 6개
print(ts_ms)
print(pd.date_range("2019-01-01",periods=6,freq="M",tz="Asia/Seoul")) # 각 달의 마지막 날을 기준으로 6개
print(pd.date_range("2019-01-01",periods=6, freq="3MS", tz="Asia/Seoul")) #3개월 간격
print(pd.date_range("2019-01-01",end=None,periods=3,freq="2H"))

DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-02-01 00:00:00+09:00',
               '2019-03-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-05-01 00:00:00+09:00', '2019-06-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='MS')
DatetimeIndex(['2019-01-31 00:00:00+09:00', '2019-02-28 00:00:00+09:00',
               '2019-03-31 00:00:00+09:00', '2019-04-30 00:00:00+09:00',
               '2019-05-31 00:00:00+09:00', '2019-06-30 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='M')
DatetimeIndex(['2019-01-01 00:00:00+09:00', '2019-04-01 00:00:00+09:00',
               '2019-07-01 00:00:00+09:00', '2019-10-01 00:00:00+09:00',
               '2020-01-01 00:00:00+09:00', '2020-04-01 00:00:00+09:00'],
              dtype='datetime64[ns, Asia/Seoul]', freq='3MS')
DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 02:00:00',
               '2019-01-01 04:00:00'],
              dtype='datetime64[ns]', freq='2H')


In [169]:
print(pd.period_range("2019-01-01",end=None,periods=3,freq="M"))
print(pd.period_range("2019-01-01",end=None,periods=3,freq="2H")) # date_range 보다 한 단위가 적게 나오는 것을 알 수 있음 -> 기간이니까


PeriodIndex(['2019-01', '2019-02', '2019-03'], dtype='period[M]', freq='M')
PeriodIndex(['2019-01-01 00:00', '2019-01-01 02:00', '2019-01-01 04:00'], dtype='period[2H]', freq='2H')


### 시계열 데이터 활용

In [172]:
# 년도, 월, 일 => 2018 / 2 / 3 개별 추출
df=pd.read_csv("./stock-data.csv")
df["new_Date"]=pd.to_datetime(df["Date"])

df["Year"]=df["new_Date"].dt.year
df["Month"]=df["new_Date"].dt.month
df["Day"]=df["new_Date"].dt.day
print(df.head())
# period로 변환하여 년, 월, 일 표기
df["Date_yr"]=df["new_Date"].dt.to_period("A") #년도까지
df["Date-m"] = df["new_Date"].dt.to_period("D") #년도,월,일까지
print(df["Date-m"])

         Date  Close  Start   High    Low  Volume   new_Date  Year  Month  Day
0  2018-07-02  10100  10850  10900  10000  137977 2018-07-02  2018      7    2
1  2018-06-29  10700  10550  10900   9990  170253 2018-06-29  2018      6   29
2  2018-06-28  10400  10900  10950  10150  155769 2018-06-28  2018      6   28
3  2018-06-27  10900  10800  11050  10500  133548 2018-06-27  2018      6   27
4  2018-06-26  10800  10900  11000  10700   63039 2018-06-26  2018      6   26
0     2018-07-02
1     2018-06-29
2     2018-06-28
3     2018-06-27
4     2018-06-26
5     2018-06-25
6     2018-06-22
7     2018-06-21
8     2018-06-20
9     2018-06-19
10    2018-06-18
11    2018-06-15
12    2018-06-14
13    2018-06-12
14    2018-06-11
15    2018-06-08
16    2018-06-07
17    2018-06-05
18    2018-06-04
19    2018-06-01
Name: Date-m, dtype: period[D]


In [179]:
# 타임스탬프 간의 날짜차이 구하기
today = pd.to_datetime("2018-12-25")
a = pd.to_datetime("2018-12-26")
b=today-a # 알아서 일수 계산해줌. 이때 timestamp와 timestamp 끼리 연산해주어야 함!!
print(b)

-1 days +00:00:00


In [203]:
3.26126e+00

3.26126